In [1]:
%pip install catboost mne colorlog

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 22.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 65.0 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 58.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


# Functional preperation

In [2]:
import pandas as pd
import numpy as np
import os
import re
import utils
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier, Pool, CatBoostRegressor

In [3]:
# test catboost
dataset = np.array([[1,4,5,6],[4,5,6,7],[30,40,50,60],[20,15,85,60]])
train_labels = [1.2,3.4,9.5,24.5]
model = CatBoostRegressor(learning_rate=1, depth=6, loss_function='RMSE')
fit_model = model.fit(dataset, train_labels)

print(fit_model.get_params())

TBB Warning: The number of workers is currently limited to 3. The request for 31 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.



0:	learn: 6.8953900	total: 46.7ms	remaining: 46.6s
1:	learn: 4.8590818	total: 47.2ms	remaining: 23.5s
2:	learn: 3.6271477	total: 47.3ms	remaining: 15.7s
3:	learn: 2.7203608	total: 47.5ms	remaining: 11.8s
4:	learn: 2.0402706	total: 47.7ms	remaining: 9.49s
5:	learn: 1.5302029	total: 47.9ms	remaining: 7.93s
6:	learn: 1.1476522	total: 48ms	remaining: 6.82s
7:	learn: 0.8607391	total: 48.2ms	remaining: 5.98s
8:	learn: 0.6455544	total: 48.4ms	remaining: 5.33s
9:	learn: 0.4841658	total: 48.6ms	remaining: 4.81s
10:	learn: 0.3631243	total: 48.8ms	remaining: 4.38s
11:	learn: 0.2723432	total: 48.9ms	remaining: 4.03s
12:	learn: 0.2042574	total: 49.1ms	remaining: 3.73s
13:	learn: 0.1531931	total: 49.3ms	remaining: 3.47s
14:	learn: 0.1148948	total: 49.5ms	remaining: 3.25s
15:	learn: 0.0861711	total: 49.6ms	remaining: 3.05s
16:	learn: 0.0646283	total: 49.8ms	remaining: 2.88s
17:	learn: 0.0484712	total: 50ms	remaining: 2.73s
18:	learn: 0.0363534	total: 50.2ms	remaining: 2.59s
19:	learn: 0.0272651	total

In [4]:
labels_df = utils.get_metadata_df("features-4", "Randomisatielijst.csv")
labels_df

,filename,procedure,patient_id,eeg_type,pre_post
0,TMS-EEG-H_07_S3_rsEEG_pre-epo.csv,itbs,07,rsEEG,pre
1,TMS-EEG-H_06_S2_rsEEG_pre-epo.csv,itbs,06,rsEEG,pre
2,TMS-EEG-H_14_S1_rsEEG_pre-epo.csv,ctbs,14,rsEEG,pre
3,TMS-EEG-H_17_S2_rsEEG_pre-epo.csv,sham,17,rsEEG,pre
4,TMS-EEG-H_16_S3_rsEEG_pre-epo.csv,itbs,16,rsEEG,pre
...,...,...,...,...,...
83,TMS-EEG-H_02_S3_rsEEG_pre-epo.csv,ctbs,02,rsEEG,pre
84,TMS-EEG-H_09_S3_rsEEG_post-epo.csv,sham,09,rsEEG,post
85,TMS-EEG-H_08_s1_rsEEG_pre-epo.csv,sham,08,rsEEG,pre
86,TMS-EEG-H_16_S2_rsEEG_post-epo.csv,sham,16,rsEEG,post


In [7]:
train_filenames, test_filenames = utils.get_train_test_split("features-4", random_state=0)
print(f"Train files: {len(train_filenames)}; Test files: {len(test_filenames)}")

Train files: 71; Test files: 17


In [22]:
def get_data(directory, filenames):
    """Returns a df containing all rows from the given feature files."""
    data = []
    for filename in filenames:
        df = pd.read_csv(os.path.join(directory, filename), header=[0,1])
        data.append(df)
    df = pd.concat(data)
    return df

def get_labels(label_name, label_df, filenames):
    """Returns a df column containg the labels for the given filenames. This is achieved by repeating the respective label for each row in the feature file."""
    labels = []
    for filename in filenames:
        df = pd.read_csv(os.path.join("features-4", filename), header=[0,1])
        rows = df.shape[0]
        filename = filename.split(".")[0]
        label = label_df[label_df['filename'] == filename][label_name].values[0]
        # add label * rows times
        labels.extend([label] * rows)
    df = pd.DataFrame(labels, columns=[label_name])
    return df

requirements = ["rsEEG"]
labels_df = pd.read_csv("labels.csv")

train_files = []
for filename in train_filenames:
    if all(x in filename for x in requirements):
        train_files.append(filename)
test_files = []
for filename in test_filenames:
    if all(x in filename for x in requirements):
        test_files.append(filename)

train_data_df = get_data("features-4", train_files)
train_labels_df = get_labels("timing", labels_df, train_files)

test_data_df = get_data("features-4", test_files)
test_labels_df = get_labels("timing", labels_df, test_files)

In [24]:
print(len(train_data_df), len(train_labels_df))

17670 17670


In [27]:
def data_prep(train_data_df, train_labels_df, test_data_df):
    def remove_outliers_zscore(df, threshold=3):
        z_scores = np.abs(zscore(df))
        return df[(z_scores < threshold).all(axis=1)]
    
    # Standardization
    scaler = StandardScaler()
    train_data_df = pd.DataFrame(scaler.fit_transform(train_data_df), columns=train_data_df.columns)
    test_data_df = pd.DataFrame(scaler.fit_transform(test_data_df), columns=test_data_df.columns)
    
    # outlier removal
    OUTLIER_THRESHOLD = 0.05
    Q1 = train_data_df.quantile(0.10)
    Q3 = train_data_df.quantile(0.90)
    IQR = Q3 - Q1

    def is_outlier(row):
        return ((row < (Q1 - 1.5 * IQR)) | (row > (Q3 + 1.5 * IQR))).sum()

    outlier_counts = train_data_df.apply(is_outlier, axis=1)
    threshold = len(train_data_df.columns) * OUTLIER_THRESHOLD
    rows_to_drop = outlier_counts[outlier_counts > threshold].index
    train_data_df_filtered = train_data_df.drop(index=rows_to_drop)
    train_labels_df = train_labels_df.drop(index=rows_to_drop)
    print(f"Original DataFrame shape: {train_data_df.shape}")
    print(f"Filtered DataFrame shape: {train_data_df_filtered.shape}")
    train_data_df = train_data_df_filtered

    return train_data_df, train_labels_df, test_data_df

train_data_df_clean, train_labels_df_clean, test_data_df_clean = data_prep(train_data_df, train_labels_df, test_data_df)

print(len(train_data_df_clean), len(train_labels_df_clean))

Original DataFrame shape: (17670, 6913)
Filtered DataFrame shape: (17665, 6913)
17665 17665


In [33]:
def train_catboost(train_data, train_labels, test_data, test_labels):    
    test_data = catboost_pool = Pool(test_data,
                                     test_labels)
    
    model = CatBoostClassifier(iterations=50,
                               depth=5,
                               learning_rate=0.2,
                               loss_function='Logloss',
                               verbose=True)
    # train the model
    model.fit(train_data, train_labels)

    # make test predictions using the resulting model
    preds_class = model.predict(test_data)
    preds_proba = model.predict_proba(test_data)
    print("class = ", preds_class)
    print("proba = ", preds_proba)
    
    accuracy = accuracy_score(test_labels_df, preds_class)
    print("Accuracy on test set: ", accuracy)
    
    return model

model = train_catboost(train_data_df_clean, train_labels_df_clean, test_data_df_clean, test_labels_df)

0:	learn: 0.6284733	total: 1.37s	remaining: 1m 7s
1:	learn: 0.5817554	total: 2.54s	remaining: 1m
2:	learn: 0.5508314	total: 3.8s	remaining: 59.5s
3:	learn: 0.5251193	total: 4.79s	remaining: 55.1s
4:	learn: 0.4933678	total: 5.86s	remaining: 52.7s
5:	learn: 0.4503451	total: 7.05s	remaining: 51.7s
6:	learn: 0.4343179	total: 8.18s	remaining: 50.3s
7:	learn: 0.4104043	total: 9.22s	remaining: 48.4s
8:	learn: 0.3870324	total: 10.3s	remaining: 47.1s
9:	learn: 0.3748858	total: 11.8s	remaining: 47.3s
10:	learn: 0.3600325	total: 13.3s	remaining: 47.1s
11:	learn: 0.3459851	total: 14.5s	remaining: 46s
12:	learn: 0.3333373	total: 15.9s	remaining: 45.3s
13:	learn: 0.3200899	total: 17.3s	remaining: 44.4s
14:	learn: 0.3118992	total: 18.8s	remaining: 43.9s
15:	learn: 0.3012737	total: 20.2s	remaining: 43s
16:	learn: 0.2900429	total: 21.6s	remaining: 42s
17:	learn: 0.2820702	total: 22.8s	remaining: 40.5s
18:	learn: 0.2723227	total: 23.9s	remaining: 39s
19:	learn: 0.2635790	total: 25.1s	remaining: 37.6s
20

## Catboost training on individual procedures

In [36]:
labels_df = utils.get_metadata_df("features-4", "Randomisatielijst.csv")
print(labels_df)

for procedure in ["itbs", "ctbs", "sham"]:
    filtered_df = labels_df[(labels_df['eeg_type'] == "rsEEG") & (labels_df['procedure'] == procedure)]
    
    # data is all files from this procedure    
    train_files = []
    for filename in train_filenames:
        if filename in filtered_df['filename'].values:
            train_files.append(filename)
    test_files = []
    for filename in test_filenames:
        if filename in filtered_df['filename'].values:
            test_files.append(filename)
    
    train_data_df = get_data("features-4", train_files)
    train_labels_df = get_labels("pre_post", labels_df, train_files)
    
    test_data_df = get_data("features-4", test_files)
    test_labels_df = get_labels("pre_post", labels_df, test_files)
    
    train_data_df, train_labels_df, test_data_df = data_prep(train_data_df, train_labels_df, test_data_df)
    model = train_catboost(train_data_df, train_labels_df, test_data_df, test_labels_df)

                              filename procedure patient_id eeg_type pre_post
0    TMS-EEG-H_07_S3_rsEEG_pre-epo.csv      itbs         07    rsEEG      pre
1    TMS-EEG-H_06_S2_rsEEG_pre-epo.csv      itbs         06    rsEEG      pre
2    TMS-EEG-H_14_S1_rsEEG_pre-epo.csv      ctbs         14    rsEEG      pre
3    TMS-EEG-H_17_S2_rsEEG_pre-epo.csv      sham         17    rsEEG      pre
4    TMS-EEG-H_16_S3_rsEEG_pre-epo.csv      itbs         16    rsEEG      pre
..                                 ...       ...        ...      ...      ...
83   TMS-EEG-H_02_S3_rsEEG_pre-epo.csv      ctbs         02    rsEEG      pre
84  TMS-EEG-H_09_S3_rsEEG_post-epo.csv      sham         09    rsEEG     post
85   TMS-EEG-H_08_s1_rsEEG_pre-epo.csv      sham         08    rsEEG      pre
86  TMS-EEG-H_16_S2_rsEEG_post-epo.csv      sham         16    rsEEG     post
87  TMS-EEG-H_10_S2_rsEEG_post-epo.csv      itbs         10    rsEEG     post

[88 rows x 5 columns]


IndexError: index 0 is out of bounds for axis 0 with size 0

## Lasso

In [38]:
# Lasso

import numpy as np
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

train_data = train_data_df
train_labels = train_labels_df

test_data = test_data_df
test_labels = test_labels_df

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train_data)
X_test_scaled = scaler.transform(test_data)

lasso = Lasso(alpha=0.7)
lasso.fit(X_train_scaled, train_labels)

y_pred_train = lasso.predict(X_train_scaled)
y_pred_test = lasso.predict(X_test_scaled)

mse_train = mean_squared_error(train_labels, y_pred_train)
mse_test = mean_squared_error(test_labels, y_pred_test)

r2_train = r2_score(train_labels, y_pred_train)
r2_test = r2_score(test_labels, y_pred_test)

print(f'Training MSE: {mse_train}')
print(f'Testing MSE: {mse_test}')
print(f'Training R^2: {r2_train}')
print(f'Testing R^2: {r2_test}')

# Assuming binary classification with threshold 0.5
threshold = 0.5
preds_class_train = (y_pred_train >= threshold).astype(int)
preds_class_test = (y_pred_test >= threshold).astype(int)

accuracy_train = accuracy_score(train_labels, preds_class_train)
accuracy_test = accuracy_score(test_labels, preds_class_test)

print(f'Training Accuracy: {accuracy_train}')
print(f'Testing Accuracy: {accuracy_test}')

Training MSE: 0.24980470052382214
Testing MSE: 0.25239942324731024
Training R^2: 0.0
Testing R^2: -0.03535254955919154
Training Accuracy: 0.5139749588971797
Testing Accuracy: 0.4211402413649605


In [58]:
# Random search
param_dist = {
    'iterations': randint(50, 200),
    'depth': randint(4, 10),
    'learning_rate': uniform(0.01, 0.3),
    'l2_leaf_reg': uniform(1, 10),
    'border_count': randint(32, 255)
}

model = CatBoostClassifier(loss_function='Logloss', verbose=0)

random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=2,
    scoring='accuracy',
    cv=5,
    random_state=42,
    verbose=1,
    n_jobs=-1
)

random_search.fit(train_data, train_labels)

print(f"Best parameters found: {random_search.best_params_}")
print(f"Best cross-validation accuracy: {random_search.best_score_}")

Fitting 5 folds for each of 2 candidates, totalling 10 fits


TBB Warning: The number of workers is currently limited to 3. The request for 31 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.

TBB Warning: The number of workers is currently limited to 3. The request for 31 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.

TBB Warning: The number of workers is currently limited to 3. The request for 31 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.

TBB Warning: The number of workers is currently limited to 3. The request for 31 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.



Best parameters found: {'border_count': 134, 'depth': 7, 'iterations': 142, 'l2_leaf_reg': 2.8343478986616377, 'learning_rate': 0.24390730008183079}
Best cross-validation accuracy: 0.5521065938129099


NameError: name 'X_test' is not defined

In [61]:
best_model = random_search.best_estimator_
y_pred = best_model.predict(test_data)
accuracy = accuracy_score(test_labels, y_pred)
print(f"Test set accuracy: {accuracy}")

Test set accuracy: 0.47503121098626716
